In [103]:
import numpy as np
import os
import tensorflow as tf
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras import regularizers, Input

In [104]:
data_dir = ('./')
class_names = ["circle", "crown", "skull", "smiley_face", "square", "star"]

In [105]:
samples_per_class = 5000

X = []
y = []

In [106]:
for label, class_name in enumerate(class_names):
    # Create the path to the .npy file (e.g., './data/apple.npy')
    file_path = os.path.join(data_dir, f"{class_name}.npy")

    # Load the .npy file → shape: (2000, 28, 28)
    data = np.load(file_path)

    X.append(data)

    # Add labels to y → [label, label, label, ..., label] (length = number of samples)
    y.append(np.full((data.shape[0]), label))


In [107]:
# Stack all image arrays vertically → final shape: (total_samples, 28, 28)
X = np.vstack(X)

# Stack all label arrays horizontally → final shape: (total_samples,)
y = np.hstack(y)

### Max Normalization

In [108]:
# Normalize pixel values to range 0–1 (from 0–255)
X = X.astype('float32') / 255.0
X.shape

(30000, 784)

In [109]:
# Flatten each image from 28x28 → 784 for dense layers
X = X.reshape(X.shape[0], -1)  # shape becomes (total_samples, 784)

In [110]:
X_train, X_, y_train, y_ = train_test_split(X,y, test_size=0.2, random_state=42, stratify=y)
# `stratify=y` ensures equal class distribution in both training and validation sets.

X_cv, X_test, y_cv, y_test = train_test_split(X_,y_, test_size=0.5, random_state=42, stratify=y_)

In [111]:
model = Sequential([ 
    Dense(256, activation='relu', input_shape=(784,), kernel_regularizer=regularizers.l2(0.001)),
    Dropout(0.3),
    Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    Dropout(0.2),
    Dense(len(class_names), activation='linear')
])

c:\Users\mehim\anaconda3\envs\ml_env\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [112]:
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
)

In [113]:
model.fit(X_train,y_train, epochs=30)

Epoch 1/30
750/750 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 1.0509
Epoch 2/30
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.5872
Epoch 3/30
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.4831
Epoch 4/30
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.4564
Epoch 5/30
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.4272
Epoch 6/30
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.4103
Epoch 7/30
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.3951
Epoch 8/30
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.3966
Epoch 9/30
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.3888
Epoch 10/30
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.3802
Epoch 11/30
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.3882
Epoch 12/30
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.3785
Epoch 13/30
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.3829
Epoch 14/30
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.3674
Epoch 15/30
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - lo

In [114]:
y_cv_pred_probs = model.predict(X_cv)   #predict probability for each label
y_cv_pred = np.argmax(y_cv_pred_probs, axis=1)  # selects the label with maximum probability

94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [115]:
print(accuracy_score(y_cv,y_cv_pred))

0.9383333333333334


In [116]:
y_test_pred_probs = model.predict(X_test) 
y_test_pred = np.argmax(y_test_pred_probs, axis=1)
print(accuracy_score(y_test,y_test_pred))

94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
0.9196666666666666


In [117]:
model.save("doodle_model.keras")